In [36]:
import pandas as pd
import requests, pandas as pd

## **Carlow**

In [37]:
file_path = '/content/rainfall_Carlow_daily.csv'

In [38]:
df = pd.read_csv(file_path, skiprows=24)

In [39]:
df['date'] = pd.to_datetime(df['date'])

/tmp/ipython-input-3532345252.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


**Data Extraction (2020-2025)**

In [40]:
start_date = '2020-01-01'
end_date = '2025-12-31'

In [41]:
carlow_rain_recent = (df['date'] >= start_date) & (df['date'] <= end_date)

In [42]:
carlow_filtered = df.loc[carlow_rain_recent]

In [43]:
carlow_filtered = carlow_filtered[['date', 'rain', 'soil', 'smd_wd', 'smd_md', 'smd_pd']]

In [44]:
print(carlow_filtered)

           date rain   soil smd_wd smd_md smd_pd
5912 2020-01-01  0.0  5.925    2.5    2.5   -5.1
5913 2020-01-02  2.2  7.728    0.9    0.9   -6.4
5914 2020-01-03  0.1  5.680    1.0    1.0   -6.0
5915 2020-01-04  0.1  6.012    1.2    1.2   -5.5
5916 2020-01-05  0.3  7.295    1.4    1.4   -5.0
...         ...  ...    ...    ...    ...    ...
8099 2025-12-27  0.0  2.266    0.5    0.5   -7.3
8100 2025-12-28  0.0  3.218    0.8    0.8   -6.6
8101 2025-12-29  0.0  3.893    1.1    1.1   -6.0
8102 2025-12-30  0.0  4.752    1.5    1.5   -5.3
8103 2025-12-31  0.0  4.356    1.8    1.8   -4.8

[2192 rows x 6 columns]


**Checking Missing Data**

In [45]:
print(carlow_filtered.isnull().sum())

date      0
rain      0
soil      0
smd_wd    0
smd_md    0
smd_pd    0
dtype: int64


**Fetching Real-time Data via API**

**1.   Checking River Codes near Carlow**

In [46]:
endp = "https://wfdapi.edenireland.ie/api/charts/catchments"

In [47]:
r = requests.get(endp, timeout = 30)
r.raise_for_status() # hTTP success or exception

In [48]:
data = r.json()
data # Count: 46 catchments

{'Success': True,
 'Count': 46,
 'Model': [{'Name': 'Foyle', 'Code': '01', 'Id': 1},
  {'Name': 'Lough Neagh & Lower Bann', 'Code': '03', 'Id': 2},
  {'Name': 'Newry, Fane, Glyde and Dee', 'Code': '06', 'Id': 3},
  {'Name': 'Boyne', 'Code': '07', 'Id': 4},
  {'Name': 'Nanny-Delvin', 'Code': '08', 'Id': 5},
  {'Name': 'Liffey and Dublin Bay', 'Code': '09', 'Id': 6},
  {'Name': 'Ovoca-Vartry', 'Code': '10', 'Id': 7},
  {'Name': 'Owenavorragh', 'Code': '11', 'Id': 8},
  {'Name': 'Slaney & Wexford Harbour', 'Code': '12', 'Id': 9},
  {'Name': 'Ballyteigue-Bannow', 'Code': '13', 'Id': 10},
  {'Name': 'Barrow', 'Code': '14', 'Id': 11},
  {'Name': 'Nore', 'Code': '15', 'Id': 12},
  {'Name': 'Suir', 'Code': '16', 'Id': 13},
  {'Name': 'Colligan-Mahon', 'Code': '17', 'Id': 14},
  {'Name': 'Blackwater (Munster)', 'Code': '18', 'Id': 15},
  {'Name': 'Lee, Cork Harbour and Youghal Bay', 'Code': '19', 'Id': 16},
  {'Name': 'Bandon-Ilen', 'Code': '20', 'Id': 17},
  {'Name': 'Dunmanus-Bantry-Kenmare',

In [49]:
type(data)

dict

In [50]:
data.keys()

dict_keys(['Success', 'Count', 'Model', 'Type'])

Barrow / Nore / Suir: key catchments in South-East

In [51]:
target = ["Barrow", "Nore", "Suir"]

In [52]:
filtered_target = [item for item in data['Model'] if item['Name'] in target]

In [53]:
filtered_target # Barrow's code: 14

[{'Name': 'Barrow', 'Code': '14', 'Id': 11},
 {'Name': 'Nore', 'Code': '15', 'Id': 12},
 {'Name': 'Suir', 'Code': '16', 'Id': 13}]

**2.   Barrow's Subcatchment**



In [57]:
endp2 = "https://wfdapi.edenireland.ie/api/catchment/14"

In [58]:
r2 = requests.get(endp2, timeout = 30)
r2.raise_for_status()

In [59]:
data2 = r2.json()
data2

{'Code': '14',
 'Name': 'Barrow',
 'Profile': 'This catchment includes the area drained by the River Barrow upstream of the River Nore confluence and all streams entering tidal water between the Barrow railway bridge at Great Island and Ringwood, Co. Kilkenny, draining a total area of 3,025km². The largest urban centre in the catchment is Carlow. The other main urban centres in this catchment are New Ross, Graiguenamanagh, Athy, Portlaoise, Mountmellick, Portarlington, Monasterevin and Kildare. The total population of the catchment is approximately 188117 with a population density of 62 people per km². The Barrow catchment is underlain in its flat northern area by limestones of varying purity which continue down the western side of the catchment and sustain good groundwater resources in places. On the eastern side of the catchment, granites dominate, culminating in the summits of the Blackstairs Mountains.',
 'Area': 3015.7,
 'GeometryExtent': '230320,114147,289600,234340',
 'LastCycle

*   The largest urban catchment in Barrow is Carlow.
*   Barrow_SC_060 has the largest area.





**3.   Water Bodies within the Barrow catchment**


In [68]:
endp3 = "https://wfdapi.edenireland.ie/api/search?v=barrow&size=100"

In [69]:
r3 = requests.get(endp3, timeout = 30)
r3.raise_for_status()

In [70]:
data3 = r3.json()
data3

{'Results': [{'Name': 'Barrow',
   'Type': 'Catchment',
   'Organisation': 'Kildare County Council',
   'Code': '14',
   'GeometryExtent': '230320,114147,289600,234340'},
  {'Name': 'Barrow_SC_030',
   'Type': 'Subcatchment',
   'Organisation': 'Offaly County Council',
   'Code': '14_1',
   'GeometryExtent': '241500,209276,258285,217280'},
  {'Name': 'Barrow_SC_140',
   'Type': 'Subcatchment',
   'Organisation': 'Carlow County Council',
   'Code': '14_10',
   'GeometryExtent': '270657,141700,284280,161060'},
  {'Name': 'Barrow_SC_020',
   'Type': 'Subcatchment',
   'Organisation': 'Laois County Council',
   'Code': '14_11',
   'GeometryExtent': '243960,189280,258270,212861'},
  {'Name': 'Barrow_SC_070',
   'Type': 'Subcatchment',
   'Organisation': 'Laois County Council',
   'Code': '14_12',
   'GeometryExtent': '257880,174109,272007,196855'},
  {'Name': 'Barrow_SC_090',
   'Type': 'Subcatchment',
   'Organisation': 'Carlow County Council',
   'Code': '14_13',
   'GeometryExtent': '270

In [72]:
results = [ r for r in data3["Results"] if r["Type"] == "River"]
results

[{'Name': 'Grand Canal Barrow Line (Barrow)',
  'Type': 'River',
  'Organisation': 'Kildare County Council',
  'Code': 'IE_14_AWB_GCBL',
  'GeometryExtent': '260682,193458,277922,225588'},
 {'Name': 'Grand Canal Milltown Feeder (Barrow)',
  'Type': 'River',
  'Organisation': 'Kildare County Council',
  'Code': 'IE_14_AWB_GCMF',
  'GeometryExtent': '274769,215500,277237,224447'},
 {'Name': 'Grand Canal Main Line East (Barrow)',
  'Type': 'River',
  'Organisation': 'Kildare County Council',
  'Code': 'IE_14_AWB_GCMLE',
  'GeometryExtent': '268758,224968,281956,230683'},
 {'Name': 'Grand Canal Main Line West (Barrow)',
  'Type': 'River',
  'Organisation': 'Kildare County Council',
  'Code': 'IE_14_AWB_GCMLW',
  'GeometryExtent': '244171,227220,254820,232047'},
 {'Name': 'BARROW_010',
  'Type': 'River',
  'Organisation': 'Laois County Council',
  'Code': 'IE_SE_14B010060',
  'GeometryExtent': '231121,204428,237216,211039'},
 {'Name': 'BARROW_020',
  'Type': 'River',
  'Organisation': 'Laoi